## Zoe Markovits, Dan Grenell 
## CSC 84020 HW2

## San Francisco Crime Data: https://www.kaggle.com/c/sf-crime/

Outline:
1. Import Dataset and Descriptive Statistics
2. Inital Deep Learning Multilayerd Perceptron
3. Training, Prediction

    A. RMS Prop Optimizer
        1. Mini batches
        2. Batch Normalization
        3. K-Cross Validation
        4. All Three
    B. RMS Prop with Momentum 
        1. Mini batches
        2. Batch Normalization
        3. K-Cross Validation
        4. All Three
    C. Choose (Adam, AdaMax, AMSGrad)
        1. Mini batches
        2. Batch Normalization
        3. K-Cross Validation
        4. All Three
4. Improve the Learning

### 1. Import Dataset and Descriptive Statistics

In [1]:
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', 40)
%matplotlib inline

In [2]:
df = pd.read_csv('/Users/zoemarkovits/Documents/Grad School/Fall 2019/Neural Networks and Deep Learning/\
HW 2/sf-crime/train.csv')

In [3]:
print(df.shape)
df.info()

(878049, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878049 entries, 0 to 878048
Data columns (total 9 columns):
Dates         878049 non-null object
Category      878049 non-null object
Descript      878049 non-null object
DayOfWeek     878049 non-null object
PdDistrict    878049 non-null object
Resolution    878049 non-null object
Address       878049 non-null object
X             878049 non-null float64
Y             878049 non-null float64
dtypes: float64(2), object(7)
memory usage: 60.3+ MB


In [4]:
from pandas_profiling import ProfileReport as pfr
pfr(df)

Number of variables,9
Number of observations,878049
Total Missing (%),0.0%
Total size in memory,60.3 MiB
Average record size in memory,72.0 B
Numeric,2
Categorical,7
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [5]:
df.head(25)

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541
5,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM UNLOCKED AUTO,Wednesday,INGLESIDE,NONE,0 Block of TEDDY AV,-122.403252,37.713431
6,2015-05-13 23:30:00,VEHICLE THEFT,STOLEN AUTOMOBILE,Wednesday,INGLESIDE,NONE,AVALON AV / PERU AV,-122.423327,37.725138
7,2015-05-13 23:30:00,VEHICLE THEFT,STOLEN AUTOMOBILE,Wednesday,BAYVIEW,NONE,KIRKWOOD AV / DONAHUE ST,-122.371274,37.727564
8,2015-05-13 23:00:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,RICHMOND,NONE,600 Block of 47TH AV,-122.508194,37.776601
9,2015-05-13 23:00:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,CENTRAL,NONE,JEFFERSON ST / LEAVENWORTH ST,-122.419088,37.807802


In [6]:
df.Category.value_counts()

LARCENY/THEFT                  174900
OTHER OFFENSES                 126182
NON-CRIMINAL                    92304
ASSAULT                         76876
DRUG/NARCOTIC                   53971
VEHICLE THEFT                   53781
VANDALISM                       44725
WARRANTS                        42214
BURGLARY                        36755
SUSPICIOUS OCC                  31414
MISSING PERSON                  25989
ROBBERY                         23000
FRAUD                           16679
FORGERY/COUNTERFEITING          10609
SECONDARY CODES                  9985
WEAPON LAWS                      8555
PROSTITUTION                     7484
TRESPASS                         7326
STOLEN PROPERTY                  4540
SEX OFFENSES FORCIBLE            4388
DISORDERLY CONDUCT               4320
DRUNKENNESS                      4280
RECOVERED VEHICLE                3138
KIDNAPPING                       2341
DRIVING UNDER THE INFLUENCE      2268
RUNAWAY                          1946
LIQUOR LAWS 

In [11]:
df = df[df['Category'].isin(['LARCENY/THEFT','OTHER OFFENSES','NON-CRIMINAL','ASSAULT',
                             'DRUG/NARCOTIC','VANDALISM','WARRANTS'])]

In [12]:
df['Datetime'] = pd.to_datetime(df['Dates'])

In [13]:
dayofweek_dummies = pd.get_dummies(df.DayOfWeek,drop_first=False).astype(int)
district_dummies = pd.get_dummies(df.PdDistrict,drop_first=False).astype(int)
resolution_dummies = pd.get_dummies(df.Resolution,drop_first=False).astype(int)
df = pd.concat([df,dayofweek_dummies,district_dummies,resolution_dummies],axis=1)  

In [14]:
df = df.drop(['Dates','DayOfWeek','PdDistrict','Resolution','Address','X','Y'], axis=1)

In [15]:
df.Category.value_counts()

LARCENY/THEFT     174900
OTHER OFFENSES    126182
NON-CRIMINAL       92304
ASSAULT            76876
DRUG/NARCOTIC      53971
VANDALISM          44725
WARRANTS           42214
Name: Category, dtype: int64

In [16]:
df['Numerical_Category'] = df['Category'].map({'LARCENY/THEFT':1,
                                             'OTHER OFFENSES':2,
                                             'NON-CRIMINAL':3,
                                             'ASSAULT':4,
                                             'DRUG/NARCOTIC':5,
                                             'VANDALISM':6,
                                             'WARRANTS':7})

In [17]:
cols = list(df.columns)
cols = [cols[-1]] + cols[:-1]
df = df[cols]

In [43]:
df.shape

(611172, 38)

In [18]:
df.head(10)

,Numerical_Category,Category,Descript,Datetime,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,BAYVIEW,CENTRAL,INGLESIDE,MISSION,NORTHERN,PARK,RICHMOND,SOUTHERN,TARAVAL,TENDERLOIN,"ARREST, BOOKED","ARREST, CITED",CLEARED-CONTACT JUVENILE FOR MORE INFO,COMPLAINANT REFUSES TO PROSECUTE,DISTRICT ATTORNEY REFUSES TO PROSECUTE,EXCEPTIONAL CLEARANCE,JUVENILE ADMONISHED,JUVENILE BOOKED,JUVENILE CITED,JUVENILE DIVERTED,LOCATED,NONE,NOT PROSECUTED,PROSECUTED BY OUTSIDE AGENCY,PROSECUTED FOR LESSER OFFENSE,PSYCHOPATHIC CASE,UNFOUNDED
0,7,WARRANTS,WARRANT ARREST,2015-05-13 23:53:00,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,2015-05-13 23:53:00,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,2015-05-13 23:33:00,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,2015-05-13 23:30:00,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,1,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,2015-05-13 23:30:00,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
5,1,LARCENY/THEFT,GRAND THEFT FROM UNLOCKED AUTO,2015-05-13 23:30:00,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
8,1,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,2015-05-13 23:00:00,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
9,1,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,2015-05-13 23:00:00,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
10,1,LARCENY/THEFT,PETTY THEFT FROM LOCKED AUTO,2015-05-13 22:58:00,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
11,2,OTHER OFFENSES,MISCELLANEOUS INVESTIGATION,2015-05-13 22:30:00,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


#### Split Data Into Train and Test

In [19]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [20]:
X = df.drop(['Numerical_Category','Category','Descript','Datetime'], axis=1)

y = df['Numerical_Category']

X, y = shuffle(X, y, random_state=1234)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [21]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

### 2. Initial Multilayered Perceptron 

In [24]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras import optimizers

In [27]:
model = Sequential()

model.add(Dense(64, input_dim = 34))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(8))
model.add(Activation('softmax'))

In [28]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [29]:
model.fit(X_train, y_train, epochs=10, verbose=1, validation_split=0.1)

Train on 385038 samples, validate on 42782 samples
Epoch 1/10
385038/385038 [==============================] - 26s 67us/step - loss: 1.5677 - acc: 0.3987 - val_loss: 1.4690 - val_acc: 0.4381
Epoch 2/10
385038/385038 [==============================] - 26s 67us/step - loss: 1.4762 - acc: 0.4364 - val_loss: 1.4535 - val_acc: 0.4412
Epoch 3/10
385038/385038 [==============================] - 26s 67us/step - loss: 1.4626 - acc: 0.4411 - val_loss: 1.4466 - val_acc: 0.4447
Epoch 4/10
385038/385038 [==============================] - 26s 68us/step - loss: 1.4570 - acc: 0.4435 - val_loss: 1.4476 - val_acc: 0.4456
Epoch 5/10
385038/385038 [==============================] - 26s 68us/step - loss: 1.4539 - acc: 0.4443 - val_loss: 1.4450 - val_acc: 0.4460
Epoch 6/10
385038/385038 [==============================] - 27s 70us/step - loss: 1.4512 - acc: 0.4447 - val_loss: 1.4422 - val_acc: 0.4472
Epoch 7/10
385038/385038 [==============================] - 27s 70us/step - loss: 1.4496 - acc: 0.4449 - val_

In [31]:
model.evaluate(X_test, y_test)

183352/183352 [==============================] - 5s 28us/step


[1.432877150597833, 0.4484488852057159]

In [32]:
from sklearn import metrics

prediction = model.predict(X_test)
prediction = np.around(prediction)

y_test_non_category = [ np.argmax(t) for t in y_test ]
y_predict_non_category = [ np.argmax(t) for t in prediction ]

from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(y_test_non_category, y_predict_non_category)

print (conf_mat)

[[    0     0     0     0     0     0     0     0]
 [50517     0  1575   164     0     0     0     0]
 [23288     0 14357   235     0     0     0     0]
 [22242     0   485  5029     0     0     0     0]
 [21788     0  1062   236     0     0     0     0]
 [14762     0  1415    40     0     0     0     0]
 [12856     0   463    91     0     0     0     0]
 [12607     0   116    24     0     0     0     0]]


### 3. Training and Prediction

### 3a. RMS Prop Optimizer

#### 3a. 1. Multilayer Perceptron with RMS Prop Optimizer and Mini-Batches

In [36]:
model = Sequential()

model.add(Dense(64, input_dim = 34))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(8))
model.add(Activation('softmax'))

In [38]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [39]:
model.fit(X_train, y_train, batch_size=64, epochs=10, verbose=1, validation_split=0.1)

Train on 385038 samples, validate on 42782 samples
Epoch 1/10
385038/385038 [==============================] - 20s 52us/step - loss: 1.6036 - acc: 0.3860 - val_loss: 1.4930 - val_acc: 0.4127
Epoch 2/10
385038/385038 [==============================] - 19s 48us/step - loss: 1.5069 - acc: 0.4138 - val_loss: 1.4668 - val_acc: 0.4360
Epoch 3/10
385038/385038 [==============================] - 20s 53us/step - loss: 1.4786 - acc: 0.4331 - val_loss: 1.4544 - val_acc: 0.4397
Epoch 4/10
385038/385038 [==============================] - 20s 51us/step - loss: 1.4676 - acc: 0.4382 - val_loss: 1.4510 - val_acc: 0.4422
Epoch 5/10
385038/385038 [==============================] - 19s 49us/step - loss: 1.4623 - acc: 0.4400 - val_loss: 1.4487 - val_acc: 0.4449
Epoch 6/10
385038/385038 [==============================] - 19s 49us/step - loss: 1.4589 - acc: 0.4412 - val_loss: 1.4473 - val_acc: 0.4459
Epoch 7/10
385038/385038 [==============================] - 19s 49us/step - loss: 1.4556 - acc: 0.4438 - val_

In [40]:
model.evaluate(X_test, y_test)

183352/183352 [==============================] - 6s 33us/step


[1.437761109991168, 0.4476635106243728]

In [41]:
from sklearn import metrics

prediction = model.predict(X_test)
prediction = np.around(prediction)

y_test_non_category = [ np.argmax(t) for t in y_test ]
y_predict_non_category = [ np.argmax(t) for t in prediction ]

from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(y_test_non_category, y_predict_non_category)

print (conf_mat)

[[    0     0     0     0     0     0     0     0]
 [50549     0  1569   138     0     0     0     0]
 [23331     0 14340   209     0     0     0     0]
 [22336     0   485  4935     0     0     0     0]
 [21793     0  1067   226     0     0     0     0]
 [14775     0  1407    35     0     0     0     0]
 [12860     0   467    83     0     0     0     0]
 [12609     0   117    21     0     0     0     0]]


#### 3a. 2. Multilayer Perceptron with RMS Prop Optimizer and Batch Normalization

In [33]:
model = Sequential()

model.add(Dense(64, input_dim = 34))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(8))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [34]:
model.fit(X_train, y_train, epochs=10, verbose=1, validation_split=0.1)

Train on 385038 samples, validate on 42782 samples
Epoch 1/10
385038/385038 [==============================] - 44s 113us/step - loss: 1.5612 - acc: 0.4130 - val_loss: 1.4542 - val_acc: 0.4446
Epoch 2/10
385038/385038 [==============================] - 34s 88us/step - loss: 1.4774 - acc: 0.4382 - val_loss: 1.4463 - val_acc: 0.4448
Epoch 3/10
385038/385038 [==============================] - 39s 102us/step - loss: 1.4667 - acc: 0.4403 - val_loss: 1.4440 - val_acc: 0.4477
Epoch 4/10
385038/385038 [==============================] - 39s 101us/step - loss: 1.4609 - acc: 0.4416 - val_loss: 1.4383 - val_acc: 0.4475
Epoch 5/10
385038/385038 [==============================] - 36s 94us/step - loss: 1.4567 - acc: 0.4429 - val_loss: 1.4345 - val_acc: 0.4477
Epoch 6/10
385038/385038 [==============================] - 39s 101us/step - loss: 1.4534 - acc: 0.4449 - val_loss: 1.4329 - val_acc: 0.4476
Epoch 7/10
385038/385038 [==============================] - 37s 97us/step - loss: 1.4515 - acc: 0.4446 - 

In [35]:
model.evaluate(X_test, y_test)

183352/183352 [==============================] - 10s 53us/step


[1.4283874150559177, 0.45030869584187794]

In [42]:
from sklearn import metrics

prediction = model.predict(X_test)
prediction = np.around(prediction)

y_test_non_category = [ np.argmax(t) for t in y_test ]
y_predict_non_category = [ np.argmax(t) for t in prediction ]

from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(y_test_non_category, y_predict_non_category)

print (conf_mat)

[[    0     0     0     0     0     0     0     0]
 [50549     0  1569   138     0     0     0     0]
 [23331     0 14340   209     0     0     0     0]
 [22336     0   485  4935     0     0     0     0]
 [21793     0  1067   226     0     0     0     0]
 [14775     0  1407    35     0     0     0     0]
 [12860     0   467    83     0     0     0     0]
 [12609     0   117    21     0     0     0     0]]


#### 3a. 3. Multilayer Perceptron with RMS Prop Optimizer and K-Cross Validation

#### 3a. 4. Multilayer Perceptron with RMS Prop Optimizer and Mini-batches, Batch Normalization, and K-Cross Validation

### 3b. RMS Prop with Momentum Optimizer (NADAM)

#### 3b. 1. Multilayer Perceptron with RMS Prop with Momentum Optimizer (NADAM) and Mini-Batches

#### 3b. 2. Multilayer Perceptron with RMS Prop with Momentum Optimizer (NADAM) and Batch Normalization

#### 3b. 3. Multilayer Perceptron with RMS Prop with Momentum Optimizer (NADAM) and K-Cross Validation

#### 3b. 4. Multilayer Perceptron with RMS Prop with Momentum Optimizer (NADAM) and Mini-batches, Batch Normalization, and K-Cross Validation

### 3c. ADAM Optimizer 

#### 3c. 1. Multilayer Perceptron with ADAM Optimizer and Mini-Batches

#### 3c. 2. Multilayer Perceptron with ADAM Optimizer and Batch Normalization

#### 3c. 3. Multilayer Perceptron with ADAM Optimizer and K-Cross Validation

#### 3c. 4. Multilayer Perceptron with ADAM Optimizer and Mini-batches, Batch Normalization, and K-Cross Validation